In [1]:
import requests
import json
from datetime import datetime
import time

base_url = "http://localhost:8000"

In [2]:
def print_response(response):
    """Pretty print API responses"""
    print(f"Status Code: {response.status_code}")
    print(f"Headers: {dict(response.headers)}")
    print("Response Body:")
    try:
        print(json.dumps(response.json(), indent=2))
    except:
        print(response.text)
    print("\n" + "="*50 + "\n")

In [3]:
# Test if the API is running
try:
    response = requests.get(f"{base_url}/docs")
    if response.status_code == 200:
        print("✅ API is running! You can view the docs at:")
        print(f"   {base_url}/docs")
    else:
        print(f"❌ API returned status code: {response.status_code}")
except requests.exceptions.ConnectionError:
    print("❌ Cannot connect to API. Make sure the backend server is running.")
    print("   Run: uvicorn main:app --reload --host 0.0.0.0 --port 8000")

❌ Cannot connect to API. Make sure the backend server is running.
   Run: uvicorn main:app --reload --host 0.0.0.0 --port 8000


In [5]:
# Test GET /users/me
print("Testing GET /users/me...")
response = requests.get(f"{base_url}/users/me")
print_response(response)

# Store user info for later tests
if response.status_code == 200:
    user_data = response.json()
    print(f"✅ User found: {user_data['name']} (ID: {user_data['id']})")
else:
    print("❌ Failed to get user information")

Testing GET /users/me...
Status Code: 200
Headers: {'date': 'Sun, 10 Aug 2025 15:21:14 GMT', 'server': 'uvicorn', 'content-length': '23', 'content-type': 'application/json'}
Response Body:
{
  "id": 1,
  "name": "Alice"
}


✅ User found: Alice (ID: 1)


In [5]:
# Test GET /threads/me
print("Testing GET /threads/me...")
response = requests.get(f"{base_url}/threads/me")
print_response(response)

# Store thread info for later tests
if response.status_code == 200:
    thread_data = response.json()
    print(f"✅ Thread found: ID {thread_data['id']}")
    print(f"   Number of messages: {len(thread_data['messages'])}")
    
    # Display existing messages
    if thread_data['messages']:
        print("\n📝 Existing messages:")
        for i, msg in enumerate(thread_data['messages'], 1):
            sender = "👤 User" if msg['is_from_user'] else "🤖 Bot"
            print(f"   {i}. {sender}: {msg['content']}")
else:
    print("❌ Failed to get thread information")

Testing GET /threads/me...
Status Code: 200
Headers: {'date': 'Sun, 10 Aug 2025 15:40:04 GMT', 'server': 'uvicorn', 'content-length': '2715', 'content-type': 'application/json'}
Response Body:
{
  "id": 1,
  "messages": [
    {
      "id": 1,
      "content": "hi",
      "is_from_user": true,
      "created_at": "2025-08-10T15:03:07.465969"
    },
    {
      "id": 2,
      "content": "I understand what you're saying. How can I help you further?",
      "is_from_user": false,
      "created_at": "2025-08-10T15:03:07.472738"
    },
    {
      "id": 3,
      "content": "whats up",
      "is_from_user": true,
      "created_at": "2025-08-10T15:08:21.931720"
    },
    {
      "id": 4,
      "content": "I understand what you're saying. How can I help you further?",
      "is_from_user": false,
      "created_at": "2025-08-10T15:08:21.938382"
    },
    {
      "id": 5,
      "content": "whats up",
      "is_from_user": true,
      "created_at": "2025-08-10T15:08:30.298928"
    },
    {
  

In [7]:
# Test POST /messages
print("Testing POST /messages...")

# Test message data
test_message = {
    "content": "Hello! This is a test message from the notebook."
}

print(f"Sending message: {test_message['content']}")
response = requests.post(
    f"{base_url}/messages",
    json=test_message,
    headers={"Content-Type": "application/json"}
)
print_response(response)

if response.status_code == 200:
    message_data = response.json()
    print("✅ Message sent successfully!")
    print(f"   User message ID: {message_data['user_message']['id']}")
    print(f"   Bot response ID: {message_data['bot_message']['id']}")
    print(f"   Bot response: {message_data['bot_message']['content']}")
else:
    print("❌ Failed to send message")

Testing POST /messages...
Sending message: Hello! This is a test message from the notebook.
Status Code: 200
Headers: {'date': 'Sun, 10 Aug 2025 15:42:11 GMT', 'server': 'uvicorn', 'content-length': '307', 'content-type': 'application/json'}
Response Body:
{
  "user_message": {
    "id": 27,
    "content": "Hello! This is a test message from the notebook.",
    "is_from_user": true,
    "created_at": "2025-08-10T15:42:11.914682"
  },
  "bot_message": {
    "id": 28,
    "content": "I appreciate you reaching out. How can I assist you today?",
    "is_from_user": false,
    "created_at": "2025-08-10T15:42:11.917074"
  }
}


✅ Message sent successfully!
   User message ID: 27
   Bot response ID: 28
   Bot response: I appreciate you reaching out. How can I assist you today?


In [8]:
# Test multiple messages
test_messages = [
    "How are you today?",
    "What's the weather like?",
    "Tell me a joke",
    "What can you help me with?"
]

print("Testing multiple messages...")
for i, message_content in enumerate(test_messages, 1):
    print(f"\n--- Message {i}: {message_content} ---")
    
    response = requests.post(
        f"{base_url}/messages",
        json={"content": message_content},
        headers={"Content-Type": "application/json"}
    )
    
    if response.status_code == 200:
        data = response.json()
        print(f"✅ User: {data['user_message']['content']}")
        print(f"🤖 Bot: {data['bot_message']['content']}")
    else:
        print(f"❌ Failed to send message {i}")
    
    # Small delay between messages
    time.sleep(0.5)

Testing multiple messages...

--- Message 1: How are you today? ---
✅ User: How are you today?
🤖 Bot: I appreciate you reaching out. How can I assist you today?

--- Message 2: What's the weather like? ---
✅ User: What's the weather like?
🤖 Bot: That's a great point. Let me think about that for a moment...

--- Message 3: Tell me a joke ---
✅ User: Tell me a joke
🤖 Bot: I understand what you're saying. How can I help you further?

--- Message 4: What can you help me with? ---
✅ User: What can you help me with?
🤖 Bot: That's a great point. Let me think about that for a moment...


In [9]:
# Verify message persistence
print("Verifying message persistence...")
response = requests.get(f"{base_url}/threads/me")

if response.status_code == 200:
    thread_data = response.json()
    print(f"✅ Thread has {len(thread_data['messages'])} total messages")
    
    print("\n📝 Complete conversation history:")
    for i, msg in enumerate(thread_data['messages'], 1):
        sender = "👤 User" if msg['is_from_user'] else "🤖 Bot"
        timestamp = msg['created_at'][:19]  # Show just date and time
        print(f"   {i:2d}. [{timestamp}] {sender}: {msg['content']}")
else:
    print("❌ Failed to verify message persistence")

Verifying message persistence...
✅ Thread has 36 total messages

📝 Complete conversation history:
    1. [2025-08-10T15:03:07] 👤 User: hi
    2. [2025-08-10T15:03:07] 🤖 Bot: I understand what you're saying. How can I help you further?
    3. [2025-08-10T15:08:21] 👤 User: whats up
    4. [2025-08-10T15:08:21] 🤖 Bot: I understand what you're saying. How can I help you further?
    5. [2025-08-10T15:08:30] 👤 User: whats up
    6. [2025-08-10T15:08:30] 🤖 Bot: I appreciate you reaching out. How can I assist you today?
    7. [2025-08-10T15:08:48] 👤 User: hola
    8. [2025-08-10T15:08:48] 🤖 Bot: I'm listening and ready to help. What's on your mind?
    9. [2025-08-10T15:10:55] 👤 User: hey hey
   10. [2025-08-10T15:10:55] 🤖 Bot: That's interesting! Tell me more about that.
   11. [2025-08-10T15:23:59] 👤 User: Hello! This is a test message from the notebook.
   12. [2025-08-10T15:23:59] 🤖 Bot: That's a great point. Let me think about that for a moment...
   13. [2025-08-10T15:25:13] 👤 User: He

In [10]:
# Test error cases
print("Testing error cases...")

# Test 1: Empty message
print("\n--- Test 1: Empty message ---")
response = requests.post(
    f"{base_url}/messages",
    json={"content": ""},
    headers={"Content-Type": "application/json"}
)
print(f"Status: {response.status_code}")
print(f"Response: {response.text}")

# Test 2: Missing content field
print("\n--- Test 2: Missing content field ---")
response = requests.post(
    f"{base_url}/messages",
    json={},
    headers={"Content-Type": "application/json"}
)
print(f"Status: {response.status_code}")
print(f"Response: {response.text}")

# Test 3: Non-existent endpoint
print("\n--- Test 3: Non-existent endpoint ---")
response = requests.get(f"{base_url}/nonexistent")
print(f"Status: {response.status_code}")
print(f"Response: {response.text}")

Testing error cases...

--- Test 1: Empty message ---
Status: 200
Response: {"user_message":{"id":37,"content":"","is_from_user":true,"created_at":"2025-08-10T15:43:34.309831"},"bot_message":{"id":38,"content":"That sounds fascinating! I'd love to hear more details.","is_from_user":false,"created_at":"2025-08-10T15:43:34.311534"}}

--- Test 2: Missing content field ---
Status: 422
Response: {"detail":[{"type":"missing","loc":["body","content"],"msg":"Field required","input":{},"url":"https://errors.pydantic.dev/2.5/v/missing"}]}

--- Test 3: Non-existent endpoint ---
Status: 404
Response: {"detail":"Not Found"}
